In [1]:
import FinanceDataReader as fdr
import pandas_datareader.data as web
from pandas_datareader import data
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import math

In [2]:
def getCloseData(ticker, start, end=None): #FinanceDataReader 에서 값 가져오기
    return fdr.DataReader(ticker, start, end)['Close']
def getCloseDataPandas(ticker, start, end=None): #PandasDataReader 에서 값 가져오기
    return data.get_data_yahoo(ticker, start, end)['Close']

In [3]:
def getDayReturn(closeDataSet): #개별 종목 일별 수익률 : 전날종가/오늘종가
    return (closeDataSet / closeDataSet.shift(1)).fillna(1)
def getCumulativeReturn(closeDataSet): #개별 종목 누적 수익률 : 자산흐름
    return closeDataSet / closeDataSet.iloc[0]


def getPortfolioResult(closeDataSet, weight=None):
    dayReturn = getDayReturn(closeDataSet)
    cumulativeReturn = getCumulativeReturn(closeDataSet)
    if not weight: # 동일 비중
        weight = [1/len(closeDataSet.columns)] * len(closeDataSet.columns)
    portfolioCumulativeReturn = (weight * cumulativeReturn).sum(axis=1)
    portfolioDayReturn = (portfolioCumulativeReturn / portfolioCumulativeReturn.shift(1)).fillna(1)
    return portfolioDayReturn, portfolioCumulativeReturn

def getEvaluation(cumulativeReturn):
    cagr = cumulativeReturn.iloc[-1] ** (252/len(cumulativeReturn))
    dd = (cumulativeReturn.cummax() - cumulativeReturn) / cumulativeReturn.cummax() * 100
    mdd= dd.max()
    print(f"최종 수익률: {cumulativeReturn.iloc[-1]}\ncagr: {cagr}\nmdd: {mdd}")
    return cagr, dd, mdd

def getRebalancingDate(closeDataSet, period="month"):
    """
    리밸런싱 일자 추출
    월별, 분기별, 연별
    """
    data = closeDataSet.copy()
    data = pd.DataFrame(data) 
    data.index = pd.to_datetime(data.index)
    data['year'] = data.index.year
    data['month'] = data.index.month
    if period == "month":
        rebalancingDate = data.drop_duplicates(['year', 'month'], keep="last").index
    if period == "quarter":
        quarter = [3,6,9,12]
        data = data.loc[data['month'].isin(quarter)]
        rebalancingDate = data.drop_duplicates(['year', 'month'], keep="last").index
    if period == "year":
        rebalancingDate = data.drop_duplicates(['year'], keep="last").index
    return rebalancingDate

def getRebalancingPortfolioResult(closeDataSet, period = "month", weightDf=None):
    """
    리밸런싱 포트폴리오 결과
    closeDataSet: 종가 데이터
    weight: 포트폴리오 개별자산 비중
    return: 포트폴리오 일간수익률, 누적수익률
    """
    if weightDf is None:
        rebalancingDate = getRebalancingDate(closeDataSet, period)
        weightDf = pd.DataFrame([[1/len(closeDataSet.columns)] * len(closeDataSet.columns)]*len(rebalancingDate),index=rebalancingDate,columns=closeDataSet.columns)
    else:
        closeDataSet = closeDataSet.loc[weightDf.index[0]:]
        rebalancingDate = getRebalancingDate(closeDataSet, period) 
        
    portfolio = pd.DataFrame() 
    totalAsset = 1 
    start = rebalancingDate[0] 
    
    for end in rebalancingDate[1:]:
        # print("*"*50)
        weight = weightDf.loc[end] # 당월 리밸런싱 비율
        priceData = closeDataSet.loc[start:end] # 당월 가격 데이터
        cumReturn = getCumulativeReturn(priceData) # 당월 누적 수익률
        weightedCumReturn = weight * cumReturn # 당월 리밸런싱 비율이 반영된 누적 수익
        netCumReturn = totalAsset * weightedCumReturn # 전월 투자 결과 반영
        start = end # start 갱신
        totalAsset = netCumReturn.iloc[-1].sum() # 총 자산 갱신
        portfolio = pd.concat([portfolio, netCumReturn]) # 매월 데이터 추가
        # print(end,weightedCumReturn)
    portfolio = portfolio.loc[~portfolio.index.duplicated(keep='last')] # 중복 데이터
    portfolioCumulativeReturn = portfolio.sum(axis=1) # 포트폴리오 누적 수익률
    portfolioDayReturn = (portfolioCumulativeReturn / portfolioCumulativeReturn.shift(1)).fillna(1)
    return portfolioDayReturn, portfolioCumulativeReturn

def getDay(str0):
    SY = int(str0[:4])
    SM = int(str0[5:7])
    SD = int(str0[8:10])
    return SY,SM,SD 

In [4]:
test =getCloseData("069500","2021-09-01") #test data
rebalDate = getRebalancingDate(test) #rebalancing_date
priceOnRebalDate = test.loc[rebalDate] #rebal_close_data
test_data = pd.DataFrame(priceOnRebalDate, columns=['Close'])

In [ ]:
# n개월 모멘텀 한번에 구하기
n= 12

for i in range(len(priceOnRebalDate)-1,-1,-1):
    tmp =0
    if i>(n-1):
        for j in range(i-1,i-n-1, -1):if priceOnRebalDate[i]>priceOnRebalDate[j] : tmp = tmp+1
      #print(i,j,priceOnRebalDate[i],priceOnRebalDate[j],tmp)
  test_data['Close'][i] = tmp/n